In [ ]:
import json
# import requests
import time
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
import os

_ = load_dotenv(find_dotenv()) # retrieve all env vars

fmp_api_key = os.getenv("FMP_API_KEY")

client = OpenAI()

